In [7]:
import os
import torch
import csv
from shutil import move
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from timeit import default_timer as timer
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/YOLO');
from CircleYolov1Module.utilities import yolo_training, map_utilities, dataset_utilities
from CircleYolov1Module.circleyolov1_structure import Yolo
from CircleYolov1Module.yolo_loss_circle import YoloLossCircleMultiBBox as Loss_fun

#HIPERPARAMETERS
BATCH_SIZE = 64
H, W = 448, 448 #Img transform size to model
LEARINING_RATE = 1e-5
EPOCHS = 250
NUM_CLASS = 1
STRIDE = 7
BBOXES = 1
LAMBDA_NOOBJ = 0.5
LAMBDA_COORD = 5
IMG_CH_NUM = 3 #number of img channels
PATH_TO_SAVE_MODEL = '/content/drive/MyDrive/!MojeYolov1/model_yolo_circle_1BBox_noobj-0,5_coord-5.pth'
DST_PATH_END_CSV = '/content/drive/MyDrive/!MojeYolov1/epoch_loss_ap_1BBox_noobj-0,5_coord-5.csv'
START_EPOCH = 1 #epoch%5 == 1
LOAD_MODEL = False
WEIGHTS_PATH = PATH_TO_SAVE_MODEL
#Paths to archives
ARH_PATH_0 = '/content/drive/MyDrive/!MojeYolov1/archiwumJabłkaParts.zip' #Arthifical Light Dataset
ARH_PATH_1 = '/content/drive/MyDrive/!MojeYolov1/archiwumMinnerApple.zip'
ARH_PATH_2 = '/content/drive/MyDrive/!MojeYolov1/archiwumACFR_multifruit-2016(apples_only).zip'

def main():
    #Seting seed for reproducibility
    torch.manual_seed(42)

    #Cheak device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    #Augumentation and normalization for imgs
    transforms = v2.Compose([
        #v2.ToImage(),
        v2.Resize(size=(H, W), antialias=True, interpolation = 3),             #3: InterpolationMode.BICUBIC
        v2.ColorJitter(brightness=(0.75, 1.75), contrast=(0.5, 1.5), saturation=(0.5, 1.5), hue=(-0.25, 0.25)),
        v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
        #v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    #Unpacking datasets
    dataset_train, dataset_val, dataset_test = dataset_utilities.unpackDatasets(
        '/content',
        transforms,
        ARH_PATH_0, ARH_PATH_1, ARH_PATH_2)

    #Loading datasets to dataloaders
    NUM_WORKERS = os.cpu_count()

    train_dataloader = DataLoader(dataset=dataset_train,
                                  batch_size=BATCH_SIZE,
                                  num_workers=NUM_WORKERS,
                                  shuffle=True)
    val_dataloader  =  DataLoader(dataset=dataset_val,
                                  batch_size=BATCH_SIZE,
                                  num_workers=NUM_WORKERS,
                                  shuffle=False)
    test_dataloader =  DataLoader(dataset=dataset_test,
                                  batch_size=BATCH_SIZE,
                                  num_workers=NUM_WORKERS,
                                  shuffle=False)

    #Seting model structure, Yolov1
    model = Yolo(
        input_shape = IMG_CH_NUM,
        num_class = NUM_CLASS,
        segmentation_stride = STRIDE,
        num_bbox = BBOXES
        ).to(device)

    #Loading model
    if LOAD_MODEL == True:
        model = Yolo(input_shape = IMG_CH_NUM,
                     num_class = NUM_CLASS,
                     segmentation_stride = STRIDE,
                     num_bbox = BBOXES
                     ).to(device)
        model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=torch.device(device)))
        model.eval()

    #Seting model checkpoints for highest mAP value
    Checkpoint = yolo_training.ModelCheckpoint(
        filepath = PATH_TO_SAVE_MODEL,
        mode = 'max',
        monitor='AP'
        )

    #Seting optimizer Adam
    optimizer = torch.optim.Adam(model.parameters(), lr = LEARINING_RATE)

    #Seting path to CSV witch will include {epoch, loss_train, mAP_val}
    path_to_CSV = '/content/epoch_loss_ap.csv'

    #Seting timer start to measure train time
    train_time_start_model = timer()

    #Function aliasing for training
    get_bboxes, mAP = map_utilities.get_bboxes, map_utilities.mean_average_precision
    print_time, train_step = yolo_training.print_train_time, yolo_training.train_step

    #Training loop
    for epoch in range(START_EPOCH, EPOCHS):
      print(f'Epoch: {(epoch):-<70}')

      loss = train_step(
                  model = model,
                  data_loader = train_dataloader,
                  yolo_loss_fn = Loss_fun(
                      C = NUM_CLASS,
                      S = STRIDE,
                      B = BBOXES,
                      lambda_noobj = LAMBDA_NOOBJ,
                      lambda_coord = LAMBDA_COORD),
                  optimizer =  optimizer,
                  device = device)

      if epoch%5 == 1:
          pred_box, true_box = get_bboxes(
                  model = model,
                  loader = val_dataloader,
                  device = device)
          # mAP 0.5:0.1:0.95
          AP =  mAP(pred_boxes=pred_box,
                    true_boxes=true_box,
                    threshold_mAP = 0.5,
                    step_threshold = 0.1,
                    stop_threshold_mAP=0.95,
                    C=1,
                    epsilon = 1E-6)

          print(f"Average Precision @0.5:0.1:0.95. For val dataset is equal: {(AP*100):.2f}%")
          Checkpoint(AP, model)
          if float(AP) >= 0.95:
              break

      train_time_end_model = timer()
      total_train_time_model = print_time(
          start = train_time_start_model,
          end = train_time_end_model,
          device = device)

      with open(path_to_CSV, mode='a', newline='') as file_csv:
          writer = csv.writer(file_csv)
          writer.writerow([epoch, loss, float(AP)])

    move('/content/epoch_loss_ap.csv', DST_PATH_END_CSV)

In [ ]:
if __name__ == "__main__":
    main()

Epoch: 1---------------------------------------------------------------------
Train loss: 861.742
Average Precision @0.5:0.1:0.95. For val dataset is equal: 0.01%
Model weights saved with AP of 5.369414066080935e-05
Train time on cuda: 495.410 seconds

Epoch: 2---------------------------------------------------------------------
Train loss: 616.716
Train time on cuda: 947.542 seconds

Epoch: 3---------------------------------------------------------------------
Train loss: 565.475
Train time on cuda: 1394.019 seconds

Epoch: 4---------------------------------------------------------------------
Train loss: 556.250
Train time on cuda: 1839.737 seconds

Epoch: 5---------------------------------------------------------------------
Train loss: 553.285
Train time on cuda: 2286.244 seconds

Epoch: 6---------------------------------------------------------------------
Train loss: 548.457
Average Precision @0.5:0.1:0.95. For val dataset is equal: 0.00%
Train time on cuda: 2777.854 seconds

Epo

In [5]:
if __name__ == "__main__":
    main()

Epoch: 186-------------------------------------------------------------------
Train loss: 40.082
Average Precision @0.5:0.1:0.95. For val dataset is equal: 32.00%
Model weights saved with AP of 0.3200072646141052
Train time on cuda: 574.729 seconds

Epoch: 187-------------------------------------------------------------------
Train loss: 34.847
Train time on cuda: 1039.483 seconds

Epoch: 188-------------------------------------------------------------------
Train loss: 30.058
Train time on cuda: 1490.156 seconds

Epoch: 189-------------------------------------------------------------------
Train loss: 26.006
Train time on cuda: 1943.850 seconds

Epoch: 190-------------------------------------------------------------------
Train loss: 23.759
Train time on cuda: 2397.542 seconds

Epoch: 191-------------------------------------------------------------------
Train loss: 25.266
Average Precision @0.5:0.1:0.95. For val dataset is equal: 24.11%
Train time on cuda: 2978.827 seconds

Epoch: 19

In [8]:
if __name__ == "__main__":
    main()

Epoch: 221-------------------------------------------------------------------
Train loss: 10.602
Average Precision @0.5:0.1:0.95. For val dataset is equal: 34.00%
Model weights saved with AP of 0.34003114700317383
Train time on cuda: 584.601 seconds

Epoch: 222-------------------------------------------------------------------
Train loss: 13.357
Train time on cuda: 1041.773 seconds

Epoch: 223-------------------------------------------------------------------
Train loss: 13.042
Train time on cuda: 1493.611 seconds

Epoch: 224-------------------------------------------------------------------
Train loss: 12.213
Train time on cuda: 1950.323 seconds

Epoch: 225-------------------------------------------------------------------
Train loss: 11.910
Train time on cuda: 2407.016 seconds

Epoch: 226-------------------------------------------------------------------
Train loss: 11.787
Average Precision @0.5:0.1:0.95. For val dataset is equal: 29.35%
Train time on cuda: 2991.927 seconds

Epoch: 2